In [1]:
!pip install 'fastai>=1.0.6'
!pip install ../input/spacy_model_en/en_core_web_sm-2.0.0.tar.gz
!python -m spacy link en_core_web_sm en
!pip install fire
from fastai import *
from fastai.text import *
from train import *

mkl-devel 2018.0.3 has requirement mkl==2018.0.3, but you'll have mkl 2019.0 which is incompatible.
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Processing /floyd/input/spacy_model_en/en_core_web_sm-2.0.0.tar.gz
  Running setup.py bdist_wheel for en-core-web-sm ... done
  Stored in directory: /root/.cache/pip/wheels/4a/17/23/da236a6b4b6deb584a9d506a0e4af2303f581106e5e91c2e64
Successfully built en-core-web-sm
mkl-devel 2018.0.3 has requirement mkl==2018.0.3, but you'll have mkl 2019.0 which is incompatible.
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which

In [2]:
!rm -rf data
# train(dir_path='data/',sample_size=1000)

In [ ]:
!floyd run "bash setup.sh && python train.py data/ --sample-size 100" --env pytorch-1.0 --gpu --follow --data frame/datasets/imdb_reviews_wt103/1:imdb_reviews_wt103
# !floyd run --task train

Creating project run. Total upload size: 93.5KiB
Syncing code ...
[================================] 99166/99166 - 00:00:00
Opening logs ...
Waiting for logs ...

2018-10-19 17:58:55,441 INFO - Preparing to run TaskInstance <TaskInstance: frame/projects/fastai_ulmfit_test/32 (id: TuPTmGgayaTFjMfG9ZAQKX)
2018-10-19 17:58:55,469 INFO - Starting attempt 1
2018-10-19 17:58:55,493 INFO - Downloading and setting up data sources
2018-10-19 17:58:55,511 INFO - Downloading and mounting dataset. ETA: 10 seconds
2018-10-19 17:58:59,539 INFO - Downloading and mounting dataset. ETA: 22 seconds
2018-10-19 17:59:28,484 INFO - Using Docker image: floydhub/pytorch:1.0.0-gpu.cuda9cudnn7-py3.36
2018-10-19 17:59:28,618 INFO - Starting container...
2018-10-19 17:59:28,838 INFO - 
################################################################################

2018-10-19 17:59:28,838 INFO - Run Output:
2018-10-19 17:59:29,430 INFO - Processing /floyd/input/spacy_model_en/en_core_web_sm-2.0.0.tar.gz
2018-10